In [ ]:
import pandas as pd
import os
from google.colab import files

def procesar_ventas_tamales():
    print("🕵️‍♂️ Iniciando escaneo de archivos y auditoría de datos...")

    # 1. DETECCIÓN DE ARCHIVOS
    # Buscamos todos los archivos CSV en el directorio actual
    archivos_csv = [f for f in os.listdir('.') if f.endswith('.csv')]

    if not archivos_csv:
        print("❌ No se encontraron archivos .csv. Por favor sube los archivos al panel de la izquierda.")
        return

    print(f"📂 Archivos detectados: {len(archivos_csv)} ({', '.join(archivos_csv)})")

    # 2. CONSOLIDACIÓN DE DATOS
    # Leemos todos los archivos y los juntamos en un solo DataFrame
    df_list = []
    for archivo in archivos_csv:
        try:
            temp_df = pd.read_csv(archivo)
            df_list.append(temp_df)
        except Exception as e:
            print(f"⚠️ Error al leer {archivo}: {e}")
            continue

    if not df_list:
        return

    df = pd.concat(df_list, ignore_index=True)
    print(f"✅ Datos consolidados: {len(df)} registros totales procesados.")

    # 3. LÓGICA DE NEGOCIO (EL CEREBRO DEL ANALISTA)
    # Diccionario maestro de mapeo: (Nombre Producto) -> (Guiso, Tipo, Factor Multiplicador)
    tamal_mapping = {
        # --- Puerco ---
        'Tamal Puerco': ('Puerco', 'Tradicional', 1),
        'Puerco': ('Puerco', 'Tradicional', 1),
        'Tamales Puerco 12pz': ('Puerco', 'Tradicional', 12),
        'Tamal Borracho Puerco': ('Puerco', 'Borracho', 1),
        'Puerco (B)': ('Puerco', 'Borracho', 1),
        'Tamal Puerco Hoja de Platano': ('Puerco', 'Hoja de Platano', 1),
        'Puerco (HP)': ('Puerco', 'Hoja de Platano', 1),

        # --- Pollo ---
        'Tamal Pollo': ('Pollo', 'Tradicional', 1),
        'Pollo': ('Pollo', 'Tradicional', 1),
        'Tamales Pollo 12pz': ('Pollo', 'Tradicional', 12),
        'Tamal Pollo Hoja de Platano': ('Pollo', 'Hoja de Platano', 1),
        'Pollo (HP)': ('Pollo', 'Hoja de Platano', 1),
        'Tamal Borracho Salsa Verde': ('Pollo Salsa Verde', 'Borracho', 1),
        'Salsa Verde (B)': ('Pollo Salsa Verde', 'Borracho', 1),
        'Tamal Borracho Mole': ('Pollo Mole', 'Borracho', 1),
        'Mole (B)': ('Pollo Mole', 'Borracho', 1),

        # --- Queso ---
        'Tamal Queso': ('Queso', 'Tradicional', 1),
        'Queso': ('Queso', 'Tradicional', 1),
        'Tamales Queso 12pz': ('Queso', 'Tradicional', 12),
        'Tamal Borracho Queso': ('Queso', 'Borracho', 1),
        'Queso (B)': ('Queso', 'Borracho', 1),

        # --- Frijol ---
        'Tamal Frijol': ('Frijol', 'Tradicional', 1),
        'Frijol': ('Frijol', 'Tradicional', 1),
        'Frijoles': ('Frijol', 'Tradicional', 1),
        'Tamales Frijoles 12pz': ('Frijol', 'Tradicional', 12),

        # --- Dulce ---
        'Tamal Dulce': ('Dulce', 'Tradicional', 1),
        'Dulce': ('Dulce', 'Tradicional', 1),
        'Tamales Dulce 12pz': ('Dulce', 'Tradicional', 12),
    }

    # 4. LIMPIEZA Y TRANSFORMACIÓN
    # Filtramos solo los productos que nos interesan (los que están en el mapa)
    df_clean = df[df['Producto_Final'].isin(tamal_mapping.keys())].copy()

    # Aplicamos el mapeo para crear nuevas columnas limpias
    # Esto separa el nombre sucio en: Guiso limpio, Tipo de tamal y Factor de cantidad
    mapping_data = df_clean['Producto_Final'].map(tamal_mapping).apply(pd.Series)
    df_clean[['Guiso', 'Tipo', 'Factor']] = mapping_data

    # CÁLCULO CRÍTICO: Multiplicamos cantidad vendida por el factor (ej. 1 paquete de 12pz = 12 tamales)
    df_clean['Unidades_Reales'] = df_clean['Cantidad'] * df_clean['Factor']

    # 5. GENERACIÓN DE TABLAS (SEGMENTACIÓN)

    # Tabla 1: Total General
    tabla_total = df_clean.groupby(['Guiso', 'Tipo'])['Unidades_Reales'].sum().reset_index()
    tabla_total.columns = ['Guiso', 'Tipo', 'Total_Unidades']
    tabla_total = tabla_total.sort_values(by='Total_Unidades', ascending=False)

    # Lógica de Paquetes: Si 'Paquete_Origen' está vacío (NaN), es venta individual. Si tiene texto, es combo.
    df_fuera = df_clean[df_clean['Paquete_Origen'].isna()]
    df_dentro = df_clean[df_clean['Paquete_Origen'].notna()]

    # Tabla 2: Fuera de Paquetes (Venta Regular)
    tabla_fuera = df_fuera.groupby(['Guiso', 'Tipo'])['Unidades_Reales'].sum().reset_index()
    tabla_fuera.columns = ['Guiso', 'Tipo', 'Unidades_Fuera_Paquete']
    tabla_fuera = tabla_fuera.sort_values(by='Unidades_Fuera_Paquete', ascending=False)

    # Tabla 3: Dentro de Paquetes (Combos)
    tabla_dentro = df_dentro.groupby(['Guiso', 'Tipo'])['Unidades_Reales'].sum().reset_index()
    tabla_dentro.columns = ['Guiso', 'Tipo', 'Unidades_Dentro_Paquete']
    tabla_dentro = tabla_dentro.sort_values(by='Unidades_Dentro_Paquete', ascending=False)

    # 6. EXPORTACIÓN AUTOMÁTICA
    nombre_salida = 'Reporte_Analisis_Tamales.xlsx'
    print("💾 Generando archivo Excel con pestañas separadas...")

    with pd.ExcelWriter(nombre_salida) as writer:
        tabla_total.to_excel(writer, sheet_name='TOTAL_GENERAL', index=False)
        tabla_fuera.to_excel(writer, sheet_name='FUERA_DE_PAQUETES', index=False)
        tabla_dentro.to_excel(writer, sheet_name='EN_COMBOS_Y_PAQUETES', index=False)

    print(f"✨ ¡Listo! Descargando {nombre_salida}...")
    files.download(nombre_salida)

# Ejecutar la función principal
if __name__ == "__main__":
    procesar_ventas_tamales()

🕵️‍♂️ Iniciando escaneo de archivos y auditoría de datos...
📂 Archivos detectados: 1 (Ventas_Diciembre_COMPLETO_V13.csv)
✅ Datos consolidados: 171716 registros totales procesados.
💾 Generando archivo Excel con pestañas separadas...
✨ ¡Listo! Descargando Reporte_Analisis_Tamales.xlsx...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>